[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RadimKozl/JLNN/blob/main/examples/Jax_lnn_base.ipynb)

# **JLNN – JAX Logical Neural Networks – Introductory example**
-----------------------------------------------------------------

This notebook shows the basic usage of the JLNN framework:
- Installation directly from GitHub
- Definition of a simple rule `0.8::A & B -> C`
- Grounding predicates on data
- Training with MSE + contradiction loss
- Saving and loading a checkpoint with structure check

**Note:** Training is stable, NaN-free, with convergence to the target interval.

## ***1. INSTALLATION AND AUTOMATIC RESTART***

In Google Colab, you just need to install the package directly from GitHub. Since the export tools are built-in, no extra parameters are needed.

In [ ]:
try:
    import jlnn
    from flax import nnx
    import jax.numpy as jnp
    print("✅ JLNN and JAX are ready.")
except ImportError:
    print("🚀 Installing JLNN from GitHub and fixing JAX for Colab...")
    # Instalace frameworku
    !pip install git+https://github.com/RadimKozl/JLNN.git
    # Fix JAX/CUDA compatibility for 2026 in Colab
    !pip install --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

    import os
    print("\n🔄 RESTARTING ENVIRONMENT... Please wait a second and then run the cell again.")
    os.kill(os.getpid(), 9)
    os.kill(os.getpid(), 9) # After this line, the cell stops and the environment restarts

🚀 Installing JLNN from GitHub and fixing JAX for Colab...
  Cloning https://github.com/RadimKozl/JLNN.git to /tmp/pip-req-build-w0ma759x
  Running command git clone --filter=blob:none --quiet https://github.com/RadimKozl/JLNN.git /tmp/pip-req-build-w0ma759x
  Resolved https://github.com/RadimKozl/JLNN.git to commit bb5667fccf8d6fa77cf68cea16f7cdd9494cb346
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of flax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


In [1]:
import os
os.environ["JAX_PLATFORMS"] = "cpu"

### ***Library imports***

In [2]:
import jax
import jax.numpy as jnp
from flax import nnx
from jlnn.symbolic.compiler import LNNFormula
from jlnn.nn.constraints import apply_constraints
from jlnn.training.losses import total_lnn_loss, logical_mse_loss, contradiction_loss
from jlnn.storage.checkpoints import save_checkpoint, load_checkpoint
import optax

In [3]:
print("JLNN loaded. JAX version:", jax.__version__)

JLNN loaded. JAX version: 0.9.0.1


## ***2. MODEL DEFINITION***



#### Initialize random numbers for gate weights

In [4]:
rngs = nnx.Rngs(42)

#### Creating a model with a simple rule

#### Formula definition:

In [5]:
formula = "0.8::A & B -> C"

#### Model creation - this internally calls JLNNCompiler and builds the graph

In [6]:
model = LNNFormula(formula, rngs)
print(f"🧩 Model compiled for formula: {formula}")

🧩 Model compiled for formula: 0.8::A & B -> C


## ***3. DATA PREPARATION***

#### Inputs for JLNN must include grounding for all symbols in the formula (including C)<br>
#### Format: [Lower Bound, Upper Bound] or [Value] for LearnedPredicate, batch size 1

In [7]:
inputs = {
    "A": jnp.array([[0.9]]),
    "B": jnp.array([[0.7]]),
    "C": jnp.array([[0.5]])   # MANDATORY – consequent must have grounding!
}

#### Target interval – we want to learn `C ≈ [0.6, 0.85]`

In [8]:
target = jnp.array([[0.6, 0.85]])

#### View initial prediction

In [9]:
initial_pred = model(inputs)
print(f"Initial prediction (before training): {initial_pred}")

Initial prediction (before training): [[1. 1.]]


## ***4. NaN-PROTECTED TRAINING + DEBUG OUTPUTS***

#### Loss function – global, pure (JAX sees it as static)

In [10]:
def loss_fn(model, inputs, target):
    pred = model(inputs)
    pred = jnp.nan_to_num(pred, nan=0.5, posinf=1.0, neginf=0.0)  # protection against NaN
    return total_lnn_loss(pred, target)

#### Optimizer definition with gradient clipping and lower LR for stability

In [11]:
optimizer = nnx.Optimizer(
    model,
    wrt=nnx.Param,
    tx=optax.chain(
        optax.clip_by_global_norm(1.0),
        optax.adam(learning_rate=0.001)
    )
)

#### Definition of training function

In [12]:
@nnx.jit
def train_step(model, optimizer, inputs, target):
    # Gradients to the model – closure is traceable (inputs/target are arrays)
    grads = nnx.grad(lambda m: loss_fn(m, inputs, target))(model)

    # Loss before update (for debug)
    loss = loss_fn(model, inputs, target)

    optimizer.update(model, grads)
    apply_constraints(model)

    final_loss = loss_fn(model, inputs, target)
    final_pred = model(inputs)

    return loss, final_loss, final_pred

#### Starting a workout

In [13]:
print("=== Starting training ===")
steps = 50
for step in range(steps):
    loss, final_loss, pred = train_step(model, optimizer, inputs, target)

    print(f"Step {step:3d} | Loss before/after constraints: {loss:.6f} → {final_loss:.6f}")
    print(f"Prediction: {pred}")
    print("─" * 60)

    if jnp.isnan(final_loss).any():
        print("❌ NaN detected! Stopping.")
        break

print("=== Training completed ===")

=== Starting training ===
Step   0 | Loss before/after constraints: 0.091250 → 0.090105
Prediction: [[0.99750346 0.99900097]]
────────────────────────────────────────────────────────────
Step   1 | Loss before/after constraints: 0.090105 → 0.089057
Prediction: [[0.9952188  0.99803764]]
────────────────────────────────────────────────────────────
Step   2 | Loss before/after constraints: 0.089057 → 0.087978
Prediction: [[0.99284816 0.99705976]]
────────────────────────────────────────────────────────────
Step   3 | Loss before/after constraints: 0.087978 → 0.086890
Prediction: [[0.9904376 0.9960748]]
────────────────────────────────────────────────────────────
Step   4 | Loss before/after constraints: 0.086890 → 0.085800
Prediction: [[0.98800725 0.9950865 ]]
────────────────────────────────────────────────────────────
Step   5 | Loss before/after constraints: 0.085800 → 0.084713
Prediction: [[0.98556733 0.9940971 ]]
────────────────────────────────────────────────────────────
Step   6 |

## ***5. RESULT AFTER TRAINING***

In [14]:
final_pred = model(inputs)
print("\nFinal prediction after training:")
print(final_pred)

print(f"\nTarget interval: {target}")
print(f"Final loss: {total_lnn_loss(final_pred, target):.6f}")


Final prediction after training:
[[0.8887138 0.9546761]]

Target interval: [[0.6  0.85]]
Final loss: 0.047156


## ***6. CHECKPOINT – SAVE AND LOAD (VERIFY)***

Saving the trained model

In [15]:
save_checkpoint(model, "trained_model.ckpt.pkl")
print("Model saved as trained_model.ckpt.pkl")

Checkpoint saved successfully: trained_model.ckpt.pkl
Model saved as trained_model.ckpt.pkl


Create a new model instance (different seed → different starting weights)

In [16]:
new_model = LNNFormula("0.8::A & B -> C", nnx.Rngs(999))

Load checkpoint

In [17]:
load_checkpoint(new_model, "trained_model.ckpt.pkl")
print("Checkpoint loaded into a new model instance.")

Model parameters successfully restored from: trained_model.ckpt.pkl
Checkpoint loaded into a new model instance.


Verification – the prediction should be the same as the original model

In [18]:
print("\nPrediction after loading checkpoint:")
print(new_model(inputs))


Prediction after loading checkpoint:
[[0.8887138 0.9546761]]


In [19]:
print("\nOriginal prediction (for comparison):")
print(model(inputs))


Original prediction (for comparison):
[[0.8887138 0.9546761]]


## ***SUMMARY***

- The model learned to approximate the target interval [0.6, 0.85] for C.
- Training was done without NaN thanks to clipping and correct LR.
- Checkpoint works – the loaded instance gives the same output.
- Next steps: experiment with different rules, larger batch, temporal operators (G/F/X).